In [1]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 09:04:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 09:04:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/22 09:04:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/09/22 09:04:55 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
inputfiles="/home/labuser/Downloads/myfiles"

In [3]:
df_sales=spark.read.csv(f"{inputfiles}/salesdata.csv")

In [4]:
df_product=spark.read.csv(f"{inputfiles}/product.csv")

In [5]:
df_sales.show()
df_product.show()

+--------------+----------+-----------+-------------+-------------------+
|           _c0|       _c1|        _c2|          _c3|                _c4|
+--------------+----------+-----------+-------------+-------------------+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|
|             1|       101|        201|            5|2023-09-22 10:15:00|
|             2|       102|        202|            3|2023-09-22 11:30:00|
|             3|       101|        203|            2|2023-09-22 12:45:00|
|             4|       103|        204|            1|2023-09-22 14:00:00|
|             5|       102|        205|            4|2023-09-22 15:15:00|
+--------------+----------+-----------+-------------+-------------------+

+----------+------------+-----------+-----+
|       _c0|         _c1|        _c2|  _c3|
+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
|       101|      Laptop|Electronics|  800|
|       102|  Smartphone|Electronics|  

In [6]:
df_sales=spark.read.option("header",True).option("inferschema",True).csv(f"{inputfiles}/salesdata.csv")
df_product=spark.read.option("header",True).option("inferschema",True).csv(f"{inputfiles}/product.csv")

In [7]:
df_sales.join(df_product, on="product_id").show()

+----------+--------------+-----------+-------------+-------------------+------------+-----------+-----+
|product_id|transaction_id|customer_id|quantity_sold|          timestamp|product_name|   category|price|
+----------+--------------+-----------+-------------+-------------------+------------+-----------+-----+
|       101|             1|        201|            5|2023-09-22 10:15:00|      Laptop|Electronics|  800|
|       102|             2|        202|            3|2023-09-22 11:30:00|  Smartphone|Electronics|  600|
|       101|             3|        203|            2|2023-09-22 12:45:00|      Laptop|Electronics|  800|
|       103|             4|        204|            1|2023-09-22 14:00:00|        Desk|  Furniture|  250|
|       102|             5|        205|            4|2023-09-22 15:15:00|  Smartphone|Electronics|  600|
+----------+--------------+-----------+-------------+-------------------+------------+-----------+-----+



In [8]:
salesprod=df_sales.join(df_product, df_sales["product_id"]==df_product["product_id"],how="left")

In [9]:
salesprod.show()

+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|product_id|product_name|   category|price|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|             1|       101|        201|            5|2023-09-22 10:15:00|       101|      Laptop|Electronics|  800|
|             2|       102|        202|            3|2023-09-22 11:30:00|       102|  Smartphone|Electronics|  600|
|             3|       101|        203|            2|2023-09-22 12:45:00|       101|      Laptop|Electronics|  800|
|             4|       103|        204|            1|2023-09-22 14:00:00|       103|        Desk|  Furniture|  250|
|             5|       102|        205|            4|2023-09-22 15:15:00|       102|  Smartphone|Electronics|  600|
+--------------+----------+-----------+-------------+-------------------

In [10]:
salesprod.filter("transaction_id=1").show()

+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|product_id|product_name|   category|price|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|             1|       101|        201|            5|2023-09-22 10:15:00|       101|      Laptop|Electronics|  800|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+



In [11]:
salesprod.where("transaction_id=1").show()

+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|product_id|product_name|   category|price|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|             1|       101|        201|            5|2023-09-22 10:15:00|       101|      Laptop|Electronics|  800|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+



In [12]:
from pyspark.sql.functions import *

In [13]:
salesprod.where(col("transaction_id")==1).show()

+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|product_id|product_name|   category|price|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|             1|       101|        201|            5|2023-09-22 10:15:00|       101|      Laptop|Electronics|  800|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+



In [14]:
employees = [(1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [15]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [16]:
employeesDF.show()

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [17]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
employeesDF.withColumn("nationality",upper(col("nationality"))).show()

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| UNITED STATES| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         INDIA|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|UNITED KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [24]:
empf=employeesDF.withColumn("ss_last_4_digit",expr('substring(ssn,-4)'))\
.withColumn("country_code",split("phone_number"," ")[0].cast("int")).\
withColumn("area_code",split("phone_number"," ")[1].cast("int"))

In [25]:
empf.show()

+-----------+----------+---------+------+--------------+----------------+-----------+---------------+------------+---------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|ss_last_4_digit|country_code|area_code|
+-----------+----------+---------+------+--------------+----------------+-----------+---------------+------------+---------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|           6789|           1|      123|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|           9123|          91|      234|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|           4444|          44|      111|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|           6118|          61|      987|
+-----------+----------+---------+------+--------------+----------------+-----------+---------------+------------+---------+


In [26]:
spark